# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [1]:
# Se importa las librerias necesirias. Para Request fue necesario instalar por terminal "pandas request"
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [2]:
#El siguiente paso es validar la consulta del HTML
html_amazon = requests.get("https://macrotrends.net/stocks/charts/AMZN/amazon/revenue", time.sleep(10)).text
#Hacer la validación para conectarse usando Chrome.
if "403 Forbidden" in html_amazon:
    headers = {"User-Agent": "Chrome/50.0.2661.102"}
    request = requests.get("https://macrotrends.net/stocks/charts/AMZN/amazon/revenue", headers = headers)
    time.sleep(10)
    html_amazon = request.text
html_amazon

'<html>\r\n<head>\r\n\n</head>\r\n<body>\r\n<div style="margin: 50px auto; width: 50%; border: 1px solid #dfdfdf; padding: 20px 50px 30px 50px; font-family:helvetica;">\r\n\r\n<h1>We do not allow automated access to our servers.</h1>\r\n<h2><p>Automated access to our data is prohibited by our data provider.</p>  \r\n<p>If you are a user attempting to access the site via a browser, please follow this process to regain access:</p>\r\n\r\n<ul>\r\n  <li>Go to <a href=\'https://whatismyipaddress.com/\' target=\'_blank\' rel=\'noopener noreferrer\'>whatismyipaddress</a> and obtain your IPv4 address</li>\r\n  <li>Email us your IPv4 address at <a href="/cdn-cgi/l/email-protection" class="__cf_email__" data-cfemail="4821262e270825292b3a273c3a2d262c3b66262d3c">[email&#160;protected]</a></li>\r\n  <li>We will add you to our whitelist within 24 hours</li>\r\n</ul>\r\n</h2>\r\n</div>\r\n<script data-cfasync="false">!function(){"use strict";function e(e){try{if("undefined"==typeof console)return;"er

In [3]:
#generamos el arbol de elementos
soup = BeautifulSoup(html_amazon,"html")
soup

<html>
<head>
</head>
<body>
<div style="margin: 50px auto; width: 50%; border: 1px solid #dfdfdf; padding: 20px 50px 30px 50px; font-family:helvetica;">
<h1>We do not allow automated access to our servers.</h1>
<h2><p>Automated access to our data is prohibited by our data provider.</p>
<p>If you are a user attempting to access the site via a browser, please follow this process to regain access:</p>
<ul>
<li>Go to <a href="https://whatismyipaddress.com/" rel="noopener noreferrer" target="_blank">whatismyipaddress</a> and obtain your IPv4 address</li>
<li>Email us your IPv4 address at <a class="__cf_email__" data-cfemail="4821262e270825292b3a273c3a2d262c3b66262d3c" href="/cdn-cgi/l/email-protection">[email protected]</a></li>
<li>We will add you to our whitelist within 24 hours</li>
</ul>
</h2>
</div>
<script data-cfasync="false">!function(){"use strict";function e(e){try{if("undefined"==typeof console)return;"error"in console?console.error(e):console.log(e)}catch(e){}}function t(e,t){v

In [4]:
#buscamos las tablas
tables = soup.find_all("historical_data_table table")  
tables

[]

In [5]:
for i, t in tables:
    if ("Amazon Quarterly Revenue" in str(t)):
        table_index = i
        break
table_index

NameError: name 'table_index' is not defined

In [6]:
# Crear el DataFrame
amazon_revenue = pd.DataFrame(columns = ["Date", "Revenue"])
for row in table[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        Date = col[0].text
        Revenue = col[1].text.replace("$", "").replace(",", "")
        amazon_revenue = pd.concat([amazon_revenue, pd.DataFrame({
            "Date": Date,
            "Revenue": Revenue
        }, index = [0])], ignore_index = True)



NameError: name 'table' is not defined